In [1]:
import nltk
import os
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity
import pandas as pd


In [2]:
def get_musicians(file):
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(file, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(),'lxml')
    musicians = []
    for ul in soup.find_all('ul'):
        for li in ul.find_all("li"):
            for a in li.find_all("a"):
                musicians.append("https://en.wikipedia.org" + a.get("href"))
                break
    zz_spot = [i[0] for i in enumerate(musicians) if "ZZ" in i[1]]
    musicians = musicians[22: zz_spot[0] + 1]
    return(musicians)

In [3]:
def get_text(file):
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(file, 'r', 'utf-8')
    results = BeautifulSoup(f, "lxml")
    all_text = ""
    for tag in results.find_all('p'):
        all_text += tag.get_text() 
    return(all_text)

In [4]:
def run():
    import pandas as pd
    def get_all_text(directory):
        import os
        all_text = []
        dictionary = {}
        for file in os.listdir(directory):
            if file[0] != ".":
                dictionary[file.split(".")[0]] = get_text(directory + "/" + file)
        return dictionary

    def conversion(adict):
        newdict = {}
        counter = 0
        for key in adict.keys():
            newdict[counter] = key
            counter += 1
        return newdict
    
    all_text = get_all_text("samples")
    test_text = get_all_text("test")
    converter = conversion(all_text)
    
    from gensim import corpora
    from gensim.models.ldamodel import LdaModel
    from gensim.parsing.preprocessing import STOPWORDS
    texts = [[word for word in text.lower().split()
        if word not in STOPWORDS and word.isalnum()]
        for text in all_text.values()]
    dictionary = corpora.Dictionary(texts)
    print(dictionary)
    corpus = [dictionary.doc2bow(text) for text in texts]
    from gensim.similarities.docsim import Similarity
    from gensim import models, similarities
    lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

    lst = []
    for key, value in test_text.items():
        doc = value
        vec_bow = dictionary.doc2bow(doc.lower().split())
        vec_lsi = lsi[vec_bow]
        index = similarities.MatrixSimilarity(lsi[corpus])
        sims = index[vec_lsi]
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        x = converter[sims[0][0]]
        y = key
        lst.append([y,x])
        
    answer = pd.DataFrame(lst, columns = ["TestName", "SampleName"])
    print(answer)
    answer.to_csv("output.csv", index=False)

In [5]:
run()

Dictionary(8673 unique tokens: ['albert', 'clifton', 'ammons', '1907', 'december']...)
                    TestName                           SampleName
0           AbuTalibmusician                           FatsDomino
1                AmosGarrett                        WayneFederman
2                 ArnettCobb                         HersalThomas
3       AsylumStreetSpankers                               DrJohn
4                     BBKing                           BobBaldori
5              BernardPurdie                          MemphisSlim
6              BigJayMcNeely                        JimJonesRevue
7               BigJoeDuskin                         BigJoeDuskin
8            BigNickNicholas                    MichaelKaeshammer
9              BillieHoliday                      BobHallmusician
10              BilliePierce                    GeneTaylorpianist
11             BillyEckstine                           FatsDomino
12         BluesianaTriangle                      Boogi